In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

from SPIworkflow.__init__ import *
from SPIworkflow.constants import *

In [2]:
def Kepler_r(M_star, P_orb):
    """Computes the orbital radius of a planet (=semimajor axis, in units of au)
       orbiting a star of mass M_star (in solar masses) and orbital period, P_orb (in days)
    """
    r_orb = (G * M_star*M_sun)**(1/3) * (P_orb*day/2/np.pi)**(2/3)/au
    return r_orb

In [3]:
print(Kepler_r(1.,365.4))

1.0003658870652499


In [4]:
def Kepler_P(M_star, r_orb):
    """Computes the orbital period (P_orb) of a planet (in days) of a planet
       orbiting a star of mass M_star (in solar masses) and semi-major axis (r_orb), in au.
    """
    
    P_orb = 2*np.pi * (G * M_star*M_sun)**(-1/2) * (r_orb*au)**(3/2)/day
    return P_orb


In [5]:
print(Kepler_P(1.,1.))

365.1995489803746
